In [ ]:
import json

name      = [ "'", 'id:'  , 'price:'  , 'size:'  , 'rooms:',    'title:'  , 'link:'  , 'address:', 'description:']    # '\n', 
new_name  = [  '"', '"id":', '"price":', '"size":', '"rooms":', '"title":', '"link":', '"address":', '"description":'] #''  ,

with open('~/real-estate-analyis/Scraper_file.txt', 'r') as file:
    data = file.read()
    for name, rename in zip(name, new_name):
        data = data.replace(name, rename)
d = json.loads(data)
#print(d)

In [ ]:
import re

price0 = [x["price"] for x in d]
#print (price0)

price1 = [x["price"].replace("(\d+)(.)(\d\d\d)(|,|.€|.+EUR)(.+)", "$1$3") for x in d]
#print(price1)


#test url: https://regex101.com/r/6zjEuc/1
price2 = [re.sub('(|.+?)(\d+)(\.)(\d\d\d)(|,|.€|.-|.+EUR)(.*$)|(Auf Anfrage)', '\g<2>\g<4>', x["price"]) for x in d]
#no price means "Auf Anfrage"

price = [int(p) if p != '' else 0 for p in price2]

#print(price)
print(len(price))

price3 = [re.sub('/\D+/g', '', x["price"]) for x in d]
#print(price2)



In [ ]:
import re
#print(d[0]["size"] )

def check_size(x):
    try:
        if (x["size"] == '--- m²' or x["size"] == 'N/A m²') :
            return '0'
        else:
            return x["size"]
    except:
        return "0"


size0 = [ check_size(x) for x in d]
#print (size0)

size1 = [re.sub("(\D+?|N/A|)(\d+)(.*)", "\g<2>", check_size(x)) for x in d]
size = [int(s) for s in size1]
#print(size)
print(len(size))

from collections import Counter
from matplotlib import pyplot as plt

myCounter = Counter(size)
print(myCounter)

#plt.rcParams["figure.figsize"] = [7.50, 3.50]
#plt.rcParams["figure.autolayout"] = True


plt.bar(myCounter.keys(), myCounter.values())

plt.show()


In [ ]:
urls = [url["link"] for url in d]
#print(urls)
print("URLS: ",len(urls))


def check_description(x):
    try:
        return x["description"]
    except:
        return "None"
    
def check_address(x):
    try:
        return x["address"]
    except:
        return "None"
    
    
addresses = [check_address(x) for x in d]
descriptions = [check_description(x) for x in d]

#print(addresses)
print("addresses: ",len(addresses))

#print(descriptions)
print("descriptions: ",len(descriptions))


In [ ]:
import pandas as pd
import plotly.graph_objs as go
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score


df = pd.DataFrame({"Size": size,
                  "Price": price,
                  "description": descriptions,
                  "address": addresses,
                  "text":  ["<a href=\"{}\">{}</a>".format(url["link"],"o") for url in d],
                  "title": [x["title"] for x in d]
                  }
                 )


import plotly.express as px


fig = px.scatter(df, x="Size", y="Price", text=df["text"], 
                 trendline="ols", 
                 hover_name="title", 
                 hover_data=["description", "address"]
                )
fig.show()